# Imports and models

In [72]:
import random
from time import time

import numpy as np
import pandas as pd

### Basic functions

In [79]:
modes = ["buy_and_hold", "risk_parity", "ts_mom", "csec_mom"]
DO_NORMALIZATION_TO_1 = True
DAYS_SHIFTED = 0
VAL_PERIOD = 0  # logic error at concatenating train and test when predicting test set with val_period > 0
MODE = modes[0]
MODE

'buy_and_hold'

In [74]:
from empyrical import (
    annual_return,
    annual_volatility,
    downside_risk,
    max_drawdown,
    sharpe_ratio,
    sortino_ratio,
    tail_ratio,
    value_at_risk,
)
from scipy.stats import kurtosis, skew


def get_data(data_config, problem_config):
    """
    Loads and splits data for each region and task into training, validation, and test sets.

    :param data_config: Dictionary containing data paths and region/task information.
    :param problem_config: Dictionary containing split configuration (val_period, holdout_period).
    :return: Dictionaries of training, validation, and test data for each region and task.
    """
    # Initialize dictionaries to hold data splits for each region and task
    Xtrain_tasks, Xval_tasks, Xtest_tasks = {}, {}, {}

    # Iterate over each region specified in the data configuration
    for region in data_config["region"]:
        # Build the list of file paths for each task in the region
        region_task_paths = [t + "_all_assets_data.pkl.gz" for t in data_config[region]]
        # Initialize nested dictionaries for each region
        Xtrain_tasks[region], Xval_tasks[region], Xtest_tasks[region] = {}, {}, {}

        # Iterate over each task and its corresponding file path
        for task_path, task in zip(region_task_paths, data_config[region]):
            # Load the pickled DataFrame for the current task
            df = pd.read_pickle(data_config["data_path"] + task_path)

            # Split the DataFrame into training, validation, and test sets
            # Training set: all rows except the last (val_period + holdout_period)
            df_train = df.iloc[
                : -(problem_config["val_period"] + problem_config["holdout_period"])
            ]
            if problem_config["val_period"] != 0:
                # Validation set: the rows just before the holdout period, of length val_period
                df_val = df.iloc[
                    -(
                        problem_config["val_period"] + problem_config["holdout_period"]
                    ) : -problem_config["holdout_period"]
                ]
            else:
                # If no validation period, validation set is same as training set
                df_val = df.iloc[
                    : -(problem_config["val_period"] + problem_config["holdout_period"])
                ]
            # Test set: the last holdout_period rows
            df_test = df.iloc[-problem_config["holdout_period"] :]

            # Convert DataFrames to numpy arrays and store in the corresponding dictionaries
            Xtrain_tasks[region][task] = df_train.values
            Xval_tasks[region][task] = df_val.values
            Xtest_tasks[region][task] = df_test.values

            # Print the region, task, and shape of the training data for verification
            print(region, task, Xtrain_tasks[region][task].shape)

    # Return the dictionaries containing the split data for all regions and tasks
    return Xtrain_tasks, Xval_tasks, Xtest_tasks


def compute_trading_costs(signal):
    # Set slippage cost per trade (currently set to zero)
    slip = 0.0005 * 0.00
    # Set basis point cost per trade (currently set to zero)
    bp = 0.0020 * 0.00
    # Calculate the absolute change in signal (trades) and multiply by total transaction cost per trade
    tc = np.abs(signal[1:, :] - signal[:-1, :]) * (bp + slip)
    # Prepend a row of zeros to align the transaction costs with the original signal shape
    tc = np.concatenate([np.zeros(signal.shape[1]).reshape(1, -1), tc], axis=0)
    # Return the transaction costs array
    return tc


def calmar_ratio(x):
    return annual_return(x).values / -max_drawdown(x)


def sharpe_ratio(x):
    return annual_return(x).values / annual_volatility(x)


def compute_performance_metrics(df_returns):
    """

    :param df_returns:
    :return:
    """

    # metrics to compute
    pf_metrics = [
        sharpe_ratio,
        calmar_ratio,
        max_drawdown,
        annual_return,
        annual_volatility,
        sortino_ratio,
        downside_risk,
        value_at_risk,
        tail_ratio,
        skew,
        kurtosis,
    ]
    pf_metrics_labels = [
        "SR",
        "CR",
        "MDD",
        "ANN_RET",
        "ANN_VOL",
        "SortR",
        "DownRisk",
        "VaR",
        "TailR",
        "Skew",
        "Kurt",
    ]

    # compute performance metric
    df_metrics = pd.DataFrame(
        index=range(df_returns.shape[1]), columns=pf_metrics_labels
    )
    # For each performance metric function (pf) and its corresponding label (pf_label),
    # apply the metric function to the returns DataFrame (df_returns).
    # Store the resulting array in the DataFrame df_metrics under the column named pf_label.
    for pf, pf_label in zip(pf_metrics, pf_metrics_labels):
        df_metrics[pf_label] = np.array(pf(df_returns))
    # Set the index of df_metrics to be the same as the column names of df_returns,
    # so that each row in df_metrics corresponds to a column (asset/strategy) in df_returns.
    df_metrics.index = df_returns.columns

    return df_metrics

### Baselines

In [75]:
def rolling_geometric_mean(returns, window):
    price_levels = 1 + returns

    # Convert to log returns
    log_returns = np.log(price_levels)

    # Calculate rolling mean in log space
    rolling_log_mean = log_returns.rolling(window=window).mean()

    # Convert back to return space
    geometric_mean = np.exp(rolling_log_mean) - 1

    return geometric_mean

In [76]:
class BuyAndHold:
    def __init__(self, x_tasks, model_config):
        # Store the training data for each task
        self.Xtrain_tasks = x_tasks
        # Path to export results or models
        self.export_path = model_config["export_path"]
        # Label for export (e.g., experiment name)
        self.export_label = model_config["export_label"]

        # List of main tasks (e.g., different datasets or asset groups)
        self.multy_task_learning_list = self.Xtrain_tasks.keys()
        # Dictionary to hold sub-task lists for each main task
        self.sub_multy_task_learning_list = {}
        # Dictionary to hold the model (signal) for each sub-task
        self.signal = {}

        # For each main task
        for task in self.multy_task_learning_list:
            # Pre-allocate dictionary for sub-tasks' models
            self.signal[task] = {}
            # List of sub-tasks for this main task
            self.sub_multy_task_learning_list[task] = self.Xtrain_tasks[task].keys()

    def train(self):
        print("Baseline models - Training is done in predict()")
        pass

    def predict(self, x_test):
        y_pred = {}
        for task in self.multy_task_learning_list:
            y_pred[task] = {}
            for sub_tk in self.sub_multy_task_learning_list[task]:
                y_pred[task][sub_tk] = np.ones_like(x_test[task][sub_tk])
                if DO_NORMALIZATION_TO_1:
                    n_assets = x_test[task][sub_tk].shape[1]
                    # Return constant signal of 1 for all assets pre-normalization
                    y_pred[task][sub_tk] *= 1 / n_assets

        return y_pred


class RiskParity:
    def __init__(self, x_tasks, model_config):
        # Store the training data for all tasks and sub-tasks
        self.Xtrain_tasks = x_tasks
        # Path to export results (if needed)
        self.export_path = model_config["export_path"]
        # Label for exported results
        self.export_label = model_config["export_label"]
        # Rolling window size for risk calculation
        self.window = model_config["risk_parity"]["window"]

        # List of main tasks (e.g., different datasets or splits)
        self.multy_task_learning_list = self.Xtrain_tasks.keys()
        # Dictionary to hold sub-task lists for each main task
        self.sub_multy_task_learning_list = {}
        # Dictionary to hold the model (signal) for each sub-task
        self.signal = {}

        # For each main task
        for task in self.multy_task_learning_list:
            # Pre-allocate dictionary for sub-tasks' models
            self.signal[task] = {}
            # List of sub-tasks for this main task
            self.sub_multy_task_learning_list[task] = self.Xtrain_tasks[task].keys()

    def train(self):
        print("Baseline models - Training is done in predict()")
        pass

    def predict(self, x_test):
        # Dictionary to hold predictions for each task and sub-task
        y_pred = {}
        # For each main task
        for task in self.multy_task_learning_list:
            y_pred[task] = {}
            # For each sub-task
            for sub_task in self.sub_multy_task_learning_list[task]:
                # Concatenate training and test data to compute rolling statistics
                x = pd.DataFrame(
                    np.concatenate(
                        [self.Xtrain_tasks[task][sub_task], x_test[task][sub_task]],
                        axis=0,
                    )
                ).shift(DAYS_SHIFTED)  # to prevent look ahead bias
                # Compute rolling standard deviation (risk) for the window size
                risk = (
                    x.rolling(window=self.window, min_periods=int(self.window / 2))
                    .std()
                    .fillna(0)  # Missing data = zero risk, zero weight
                    .values[-x_test[task][sub_task].shape[0] :, :]
                )

                # Calculate inverse risk (will produce 0 for zero risk)
                inverse_risk = np.where(risk == 0, 0.0, 1.0 / risk)

                # Compute risk parity weights: inverse risk, normalized to sum to 1 across assets
                sum_inverse_risk = np.sum(inverse_risk, axis=1).reshape(-1, 1)
                y_pred[task][sub_task] = np.where(
                    sum_inverse_risk == 0,
                    0.0,
                    inverse_risk / np.repeat(sum_inverse_risk, risk.shape[1], axis=1),
                )
                if not DO_NORMALIZATION_TO_1:
                    n_assets = x_test[task][sub_task].shape[1]
                    y_pred[task][sub_task] *= n_assets

        # Return the dictionary of predictions (risk parity weights) for all tasks and sub-tasks
        return y_pred


class TimeSeriesMomentum:
    def __init__(self, x_tasks, model_config):
        # Store the training data for all tasks and sub-tasks
        self.Xtrain_tasks = x_tasks
        # Path and label for exporting results (not used in this baseline)
        self.export_path = model_config["export_path"]
        self.export_label = model_config["export_label"]
        # Rolling window size for momentum calculation
        self.window = model_config["ts_mom"]["window"]

        # List of main tasks (e.g., different datasets or splits)
        self.multy_task_learning_list = self.Xtrain_tasks.keys()
        # Dictionary to store sub-tasks for each main task
        self.sub_multy_task_learning_list = {}
        # Dictionary to store dummy LinearRegression models for each sub-task (not used for prediction)
        self.signal = {}

        for task in self.multy_task_learning_list:
            # Initialize dictionary for sub-tasks under this main task
            self.signal[task] = {}
            # List of sub-tasks for this main task
            self.sub_multy_task_learning_list[task] = self.Xtrain_tasks[task].keys()

    def train(self):
        # No training is performed for this baseline model; prediction is rule-based
        print("Baseline models - Training is done in predict()")
        pass

    def predict(self, x_test):
        # Dictionary to hold predictions for each task and sub-task
        y_pred = {}
        # For each main task
        for task in self.multy_task_learning_list:
            y_pred[task] = {}
            # For each sub-task
            for sub_task in self.sub_multy_task_learning_list[task]:
                # Concatenate training and test data to compute rolling mean over the full series
                x = pd.DataFrame(
                    np.concatenate(
                        [self.Xtrain_tasks[task][sub_task], x_test[task][sub_task]],
                        axis=0,
                    )
                ).shift(DAYS_SHIFTED)  # to prevent look ahead bias

                # Calculate geometric mean
                geometric_mean = rolling_geometric_mean(x, self.window)
                # Take negative for contrarian strategy
                contrarian_signal = -geometric_mean.values[
                    -x_test[task][sub_task].shape[0] :, :
                ]

                # Take negative for contrarian strategy
                y_pred[task][sub_task] = contrarian_signal
                # if DO_NORMALIZATION:
                abs_signal = np.abs(contrarian_signal)
                sum_abs = np.sum(abs_signal, axis=1, keepdims=True)

                y_pred[task][sub_task] = np.where(
                    sum_abs == 0, 0.0, contrarian_signal / sum_abs
                )
                if not DO_NORMALIZATION_TO_1:
                    n_assets = x_test[task][sub_task].shape[1]
                    y_pred[task][sub_task] *= n_assets

        # Return the dictionary of predictions for all tasks and sub-tasks
        return y_pred


class CrossSectionalMomentum:
    def __init__(self, x_tasks, model_config):
        # Store the training data for all tasks and sub-tasks
        self.Xtrain_tasks = x_tasks
        # Path to export results (if needed)
        self.export_path = model_config["export_path"]
        # Label for exported results
        self.export_label = model_config["export_label"]
        # Rolling window size for momentum calculation
        self.window = model_config["csec_mom"]["window"]
        # Fraction of assets to select for long/short positions
        self.fraction = model_config["csec_mom"]["fraction"]

        # List of main tasks (e.g., regions or datasets)
        self.multy_task_learning_list = self.Xtrain_tasks.keys()
        # Dictionary to store sub-tasks for each main task
        self.sub_multy_task_learning_list = {}
        # Dictionary to store dummy LinearRegression models for each sub-task (not used for prediction)
        self.signal = {}

        for task in self.multy_task_learning_list:
            # Initialize dictionary for sub-tasks under this main task
            self.signal[task] = {}
            # List of sub-tasks for this main task
            self.sub_multy_task_learning_list[task] = self.Xtrain_tasks[task].keys()

    def train(self):
        # No training is performed for this baseline model; prediction is rule-based
        print("Baseline models - Training is done in predict()")
        pass

    def predict(self, x_test):
        # Dictionary to hold predictions for each task and sub-task
        y_pred = {}
        # For each main task
        for task in self.multy_task_learning_list:
            y_pred[task] = {}
            # For each sub-task
            for sub_task in self.sub_multy_task_learning_list[task]:
                # Concatenate training and test data to compute rolling mean over the full series
                x = pd.DataFrame(
                    np.concatenate(
                        [self.Xtrain_tasks[task][sub_task], x_test[task][sub_task]],
                        axis=0,
                    )
                ).shift(DAYS_SHIFTED)  # Shift to prevent look-ahead bias

                # Compute rolling mean (momentum signal) over the specified window
                geometric_mean = rolling_geometric_mean(x, self.window)
                signal = geometric_mean.values[-x_test[task][sub_task].shape[0] :, :]

                # Rank the signal cross-sectionally (across assets) for each time point
                # Divide by number of assets to get quantile ranks
                ranks = pd.DataFrame(signal).rank(axis=1) / signal.shape[1]

                # Identify bottom fraction (to short) and top fraction (to long)
                bottom = ranks.values < self.fraction
                top = ranks.values > (1 - self.fraction)

                # Assign positions: -signal for both top and bottom (contrarian strategy)
                # Only assets in top or bottom fraction get nonzero positions
                crosssectional_signal = np.multiply(-signal, (bottom + top))

                abs_signal = np.abs(crosssectional_signal)
                sum_abs = np.sum(abs_signal, axis=1, keepdims=True)

                y_pred[task][sub_task] = np.where(
                    sum_abs == 0, 0.0, crosssectional_signal / sum_abs
                )
                if not DO_NORMALIZATION_TO_1:
                    n_assets = signal.shape[1]
                    y_pred[task][sub_task] *= n_assets

        # Return the dictionary of predictions for all tasks and sub-tasks
        return y_pred

# LOOP

In [77]:
# reproducibility params
manualSeed = 999999999
np.random.seed(manualSeed)
random.seed(manualSeed)

# data params
data_config = {
    "data_path": ".\\Tasks\\",
    "region": ["Asia and Pacific", "Europe", "Americas", "MEA"],
    "Europe": [
        "Europe_AEX",
        "Europe_ASE",
        "Europe_ATX",
        "Europe_BEL20",
        "Europe_BUX",
        "Europe_BVLX",
        "Europe_CAC",
        "Europe_CYSMMAPA",
        "Europe_DAX",
        "Europe_HEX",
        "Europe_IBEX",
        "Europe_ISEQ",
        "Europe_KFX",
        "Europe_OBX",
        "Europe_OMX",
        "Europe_SMI",
        "Europe_UKX",
        "Europe_VILSE",
        "Europe_WIG20",
        "Europe_XU100",
        "Europe_SOFIX",
        "Europe_SBITOP",
        "Europe_PX",
        "Europe_CRO",
    ],
    "Asia and Pacific": [
        "Asia and Pacific_AS51",
        "Asia and Pacific_FBMKLCI",
        "Asia and Pacific_HSI",
        "Asia and Pacific_JCI",
        "Asia and Pacific_KOSPI",
        "Asia and Pacific_KSE100",
        "Asia and Pacific_NIFTY",
        "Asia and Pacific_NKY",
        "Asia and Pacific_NZSE50FG",
        "Asia and Pacific_PCOMP",
        "Asia and Pacific_STI",
        "Asia and Pacific_SHSZ300",
        "Asia and Pacific_TWSE",
    ],
    "Americas": [
        "Americas_IBOV",
        "Americas_MEXBOL",
        "Americas_MERVAL",
        "Americas_SPTSX",
        "Americas_SPX",
        "Americas_RTY",
    ],
    "MEA": [
        "MEA_DFMGI",
        "MEA_DSM",
        "MEA_EGX30",
        "MEA_FTN098",
        "MEA_JOSMGNFF",
        "MEA_KNSMIDX",
        "MEA_KWSEPM",
        "MEA_MOSENEW",
        "MEA_MSM30",
        "MEA_NGSE30",
        "MEA_PASISI",
        "MEA_SASEIDX",
        "MEA_SEMDEX",
        "MEA_TA-35",
        "MEA_TOP40",
    ],
    "additional_data_path": "_all_assets_data.pkl.gz",
}

# problem params
problem_config = {
    "export_path": "./Results/",
    "val_period": VAL_PERIOD,  # if val is 0, then its results are the same as training
    "holdout_period": 756,
}


In [80]:
for MODE in modes:
    for DAYS_SHIFTED in [0, 1]:
        # model params
        model_config = {
            "baseline": MODE,
            "buy_and_hold": {},
            "risk_parity": {"window": 252},
            "ts_mom": {"window": 252},
            "csec_mom": {"window": 252, "fraction": 0.33},
        }

        # pre-allocation
        export_label = (
            "valperiod_"
            + str(problem_config["val_period"])
            + "_testperiod_"
            + str(problem_config["holdout_period"])
            + "_baseline_"
            + model_config["baseline"]
        )
        if DAYS_SHIFTED:
            export_label += "_shifted_" + str(DAYS_SHIFTED)

        data_config["export_label"] = export_label
        problem_config["export_label"] = export_label
        model_config["export_label"] = export_label
        model_config["export_path"] = problem_config["export_path"]

        # get data
        Xtrain_tasks, Xval_tasks, Xtest_tasks = get_data(data_config, problem_config)
        ### Training
        # set model
        if model_config["baseline"] == "buy_and_hold":
            trad_strat = BuyAndHold(Xtrain_tasks, model_config)
            add_label = [""] * len(data_config["region"])

        elif model_config["baseline"] == "risk_parity":
            trad_strat = RiskParity(Xtrain_tasks, model_config)
            add_label = [""] * len(data_config["region"])

        elif model_config["baseline"] == "ts_mom":
            trad_strat = TimeSeriesMomentum(Xtrain_tasks, model_config)
            add_label = [""] * len(data_config["region"])

        elif model_config["baseline"] == "csec_mom":
            trad_strat = CrossSectionalMomentum(Xtrain_tasks, model_config)
            add_label = [""] * len(data_config["region"])

        # additional labelling
        to_add_label = {}
        for lab, region in zip(add_label, data_config["region"]):
            to_add_label[region] = lab

        # train model
        start = time()
        trad_strat.train()
        print(time() - start)

        # get signals
        Xtrain_signal = trad_strat.predict(Xtrain_tasks)
        Xval_signal = trad_strat.predict(Xval_tasks)
        Xtest_signal = trad_strat.predict(Xtest_tasks)
        ### Evaluation
        # compute results
        k, g = True, True
        for region in data_config["region"]:
            region_task_paths = [
                t + "_all_assets_data.pkl.gz" for t in data_config[region]
            ]

            z = True
            for task, task_path in zip(data_config[region], region_task_paths):
                # get signal
                pred_train = Xtrain_signal[region][task][:-1, :]
                pred_val = Xval_signal[region][task][:-1, :]
                pred_test = Xtest_signal[region][task][:-1, :]

                # get target
                Ytrain = Xtrain_tasks[region][task][1:, :]
                Yval = Xval_tasks[region][task][1:, :]
                Ytest = Xtest_tasks[region][task][1:, :]

                # compute returns
                df_train_ret = np.multiply(pred_train, Ytrain) - compute_trading_costs(
                    pred_train
                )
                df_val_ret = np.multiply(pred_val, Yval) - compute_trading_costs(
                    pred_val
                )
                df_test_ret = np.multiply(pred_test, Ytest) - compute_trading_costs(
                    pred_test
                )

                # get performance metrics
                df = pd.read_pickle(data_config["data_path"] + task_path)
                df_train_ret = pd.DataFrame(df_train_ret, columns=df.columns)
                df_train_metrics = compute_performance_metrics(df_train_ret)
                df_train_metrics["exchange"] = task

                df_val_ret = pd.DataFrame(df_val_ret, columns=df.columns)
                df_val_metrics = compute_performance_metrics(df_val_ret)
                df_val_metrics["exchange"] = task

                df_test_ret = pd.DataFrame(df_test_ret, columns=df.columns)
                df_test_metrics = compute_performance_metrics(df_test_ret)
                df_test_metrics["exchange"] = task

                if z:
                    all_df_train_metrics = df_train_metrics.copy()
                    all_df_val_metrics = df_val_metrics.copy()
                    all_df_test_metrics = df_test_metrics.copy()
                    z = False
                else:
                    all_df_train_metrics = pd.concat(
                        [all_df_train_metrics, df_train_metrics], axis=0
                    )
                    all_df_val_metrics = pd.concat(
                        [all_df_val_metrics, df_val_metrics], axis=0
                    )
                    all_df_test_metrics = pd.concat(
                        [all_df_test_metrics, df_test_metrics], axis=0
                    )

            # export results
            all_df_train_metrics["region"] = region
            all_df_train_metrics["set"] = "train"
            all_df_val_metrics["region"] = region
            all_df_val_metrics["set"] = "val"
            all_df_test_metrics["region"] = region
            all_df_test_metrics["set"] = "test"

            pd.concat(
                [all_df_train_metrics, all_df_val_metrics, all_df_test_metrics], axis=0
            ).to_csv(
                problem_config["export_path"]
                + region
                + "_"
                + problem_config["export_label"]
                + to_add_label[region]
                + ".csv"
            )

            # consolidate test results
            if g:
                global_df_test_metrics = all_df_test_metrics.copy()
                g = False
            else:
                global_df_test_metrics = pd.concat(
                    [global_df_test_metrics, all_df_test_metrics.copy()], axis=0
                )

Asia and Pacific Asia and Pacific_AS51 (5623, 86)
Asia and Pacific Asia and Pacific_FBMKLCI (5623, 22)
Asia and Pacific Asia and Pacific_HSI (5623, 37)
Asia and Pacific Asia and Pacific_JCI (5623, 44)
Asia and Pacific Asia and Pacific_KOSPI (5623, 297)
Asia and Pacific Asia and Pacific_KSE100 (5623, 41)
Asia and Pacific Asia and Pacific_NIFTY (5623, 37)
Asia and Pacific Asia and Pacific_NKY (5623, 186)
Asia and Pacific Asia and Pacific_NZSE50FG (5623, 19)
Asia and Pacific Asia and Pacific_PCOMP (5623, 16)
Asia and Pacific Asia and Pacific_STI (5623, 25)
Asia and Pacific Asia and Pacific_SHSZ300 (5623, 18)
Asia and Pacific Asia and Pacific_TWSE (5623, 227)
Europe Europe_AEX (5623, 17)
Europe Europe_ASE (5623, 51)
Europe Europe_ATX (5623, 13)
Europe Europe_BEL20 (5623, 14)
Europe Europe_BUX (5623, 8)
Europe Europe_BVLX (5623, 17)
Europe Europe_CAC (5623, 35)
Europe Europe_CYSMMAPA (5623, 42)
Europe Europe_DAX (5623, 25)
Europe Europe_HEX (5623, 57)
Europe Europe_IBEX (5623, 23)
Europe Eu

C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide


Asia and Pacific Asia and Pacific_AS51 (5623, 86)
Asia and Pacific Asia and Pacific_FBMKLCI (5623, 22)
Asia and Pacific Asia and Pacific_HSI (5623, 37)
Asia and Pacific Asia and Pacific_JCI (5623, 44)
Asia and Pacific Asia and Pacific_KOSPI (5623, 297)
Asia and Pacific Asia and Pacific_KSE100 (5623, 41)
Asia and Pacific Asia and Pacific_NIFTY (5623, 37)
Asia and Pacific Asia and Pacific_NKY (5623, 186)
Asia and Pacific Asia and Pacific_NZSE50FG (5623, 19)
Asia and Pacific Asia and Pacific_PCOMP (5623, 16)
Asia and Pacific Asia and Pacific_STI (5623, 25)
Asia and Pacific Asia and Pacific_SHSZ300 (5623, 18)
Asia and Pacific Asia and Pacific_TWSE (5623, 227)
Europe Europe_AEX (5623, 17)
Europe Europe_ASE (5623, 51)
Europe Europe_ATX (5623, 13)
Europe Europe_BEL20 (5623, 14)
Europe Europe_BUX (5623, 8)
Europe Europe_BVLX (5623, 17)
Europe Europe_CAC (5623, 35)
Europe Europe_CYSMMAPA (5623, 42)
Europe Europe_DAX (5623, 25)
Europe Europe_HEX (5623, 57)
Europe Europe_IBEX (5623, 23)
Europe Eu

C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide


Asia and Pacific Asia and Pacific_AS51 (5623, 86)
Asia and Pacific Asia and Pacific_FBMKLCI (5623, 22)
Asia and Pacific Asia and Pacific_HSI (5623, 37)
Asia and Pacific Asia and Pacific_JCI (5623, 44)
Asia and Pacific Asia and Pacific_KOSPI (5623, 297)
Asia and Pacific Asia and Pacific_KSE100 (5623, 41)
Asia and Pacific Asia and Pacific_NIFTY (5623, 37)
Asia and Pacific Asia and Pacific_NKY (5623, 186)
Asia and Pacific Asia and Pacific_NZSE50FG (5623, 19)
Asia and Pacific Asia and Pacific_PCOMP (5623, 16)
Asia and Pacific Asia and Pacific_STI (5623, 25)
Asia and Pacific Asia and Pacific_SHSZ300 (5623, 18)
Asia and Pacific Asia and Pacific_TWSE (5623, 227)
Europe Europe_AEX (5623, 17)
Europe Europe_ASE (5623, 51)
Europe Europe_ATX (5623, 13)
Europe Europe_BEL20 (5623, 14)
Europe Europe_BUX (5623, 8)
Europe Europe_BVLX (5623, 17)
Europe Europe_CAC (5623, 35)
Europe Europe_CYSMMAPA (5623, 42)
Europe Europe_DAX (5623, 25)
Europe Europe_HEX (5623, 57)
Europe Europe_IBEX (5623, 23)
Europe Eu

...

C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\3297836661.py:170: RuntimeWarning: invalid value encountered in divide
  sum_abs == 0, 0.0, contrarian_signal / sum_abs
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  retu

Asia and Pacific Asia and Pacific_AS51 (5623, 86)
Asia and Pacific Asia and Pacific_FBMKLCI (5623, 22)
Asia and Pacific Asia and Pacific_HSI (5623, 37)
Asia and Pacific Asia and Pacific_JCI (5623, 44)
Asia and Pacific Asia and Pacific_KOSPI (5623, 297)
Asia and Pacific Asia and Pacific_KSE100 (5623, 41)
Asia and Pacific Asia and Pacific_NIFTY (5623, 37)
Asia and Pacific Asia and Pacific_NKY (5623, 186)
Asia and Pacific Asia and Pacific_NZSE50FG (5623, 19)
Asia and Pacific Asia and Pacific_PCOMP (5623, 16)
Asia and Pacific Asia and Pacific_STI (5623, 25)
Asia and Pacific Asia and Pacific_SHSZ300 (5623, 18)
Asia and Pacific Asia and Pacific_TWSE (5623, 227)
Europe Europe_AEX (5623, 17)
Europe Europe_ASE (5623, 51)
Europe Europe_ATX (5623, 13)
Europe Europe_BEL20 (5623, 14)
Europe Europe_BUX (5623, 8)
Europe Europe_BVLX (5623, 17)
Europe Europe_CAC (5623, 35)
Europe Europe_CYSMMAPA (5623, 42)
Europe Europe_DAX (5623, 25)
Europe Europe_HEX (5623, 57)
Europe Europe_IBEX (5623, 23)
Europe Eu

C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\3297836661.py:247: RuntimeWarning: invalid value encountered in divide
  sum_abs == 0, 0.0, crosssectional_signal / sum_abs
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  

Asia and Pacific Asia and Pacific_AS51 (5623, 86)
Asia and Pacific Asia and Pacific_FBMKLCI (5623, 22)
Asia and Pacific Asia and Pacific_HSI (5623, 37)
Asia and Pacific Asia and Pacific_JCI (5623, 44)
Asia and Pacific Asia and Pacific_KOSPI (5623, 297)
Asia and Pacific Asia and Pacific_KSE100 (5623, 41)
Asia and Pacific Asia and Pacific_NIFTY (5623, 37)
Asia and Pacific Asia and Pacific_NKY (5623, 186)
Asia and Pacific Asia and Pacific_NZSE50FG (5623, 19)
Asia and Pacific Asia and Pacific_PCOMP (5623, 16)
Asia and Pacific Asia and Pacific_STI (5623, 25)
Asia and Pacific Asia and Pacific_SHSZ300 (5623, 18)
Asia and Pacific Asia and Pacific_TWSE (5623, 227)
Europe Europe_AEX (5623, 17)
Europe Europe_ASE (5623, 51)
Europe Europe_ATX (5623, 13)
Europe Europe_BEL20 (5623, 14)
Europe Europe_BUX (5623, 8)
Europe Europe_BVLX (5623, 17)
Europe Europe_CAC (5623, 35)
Europe Europe_CYSMMAPA (5623, 42)
Europe Europe_DAX (5623, 25)
Europe Europe_HEX (5623, 57)
Europe Europe_IBEX (5623, 23)
Europe Eu

C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\3297836661.py:247: RuntimeWarning: invalid value encountered in divide
  sum_abs == 0, 0.0, crosssectional_signal / sum_abs
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  

# Separate

### Main loop configuration

In [59]:
MODE = modes[3]
MODE

'csec_mom'

In [60]:
# reproducibility params
manualSeed = 999999999
np.random.seed(manualSeed)
random.seed(manualSeed)

# data params
data_config = {
    "data_path": ".\\Tasks\\",
    "region": ["Asia and Pacific", "Europe", "Americas", "MEA"],
    "Europe": [
        "Europe_AEX",
        "Europe_ASE",
        "Europe_ATX",
        "Europe_BEL20",
        "Europe_BUX",
        "Europe_BVLX",
        "Europe_CAC",
        "Europe_CYSMMAPA",
        "Europe_DAX",
        "Europe_HEX",
        "Europe_IBEX",
        "Europe_ISEQ",
        "Europe_KFX",
        "Europe_OBX",
        "Europe_OMX",
        "Europe_SMI",
        "Europe_UKX",
        "Europe_VILSE",
        "Europe_WIG20",
        "Europe_XU100",
        "Europe_SOFIX",
        "Europe_SBITOP",
        "Europe_PX",
        "Europe_CRO",
    ],
    "Asia and Pacific": [
        "Asia and Pacific_AS51",
        "Asia and Pacific_FBMKLCI",
        "Asia and Pacific_HSI",
        "Asia and Pacific_JCI",
        "Asia and Pacific_KOSPI",
        "Asia and Pacific_KSE100",
        "Asia and Pacific_NIFTY",
        "Asia and Pacific_NKY",
        "Asia and Pacific_NZSE50FG",
        "Asia and Pacific_PCOMP",
        "Asia and Pacific_STI",
        "Asia and Pacific_SHSZ300",
        "Asia and Pacific_TWSE",
    ],
    "Americas": [
        "Americas_IBOV",
        "Americas_MEXBOL",
        "Americas_MERVAL",
        "Americas_SPTSX",
        "Americas_SPX",
        "Americas_RTY",
    ],
    "MEA": [
        "MEA_DFMGI",
        "MEA_DSM",
        "MEA_EGX30",
        "MEA_FTN098",
        "MEA_JOSMGNFF",
        "MEA_KNSMIDX",
        "MEA_KWSEPM",
        "MEA_MOSENEW",
        "MEA_MSM30",
        "MEA_NGSE30",
        "MEA_PASISI",
        "MEA_SASEIDX",
        "MEA_SEMDEX",
        "MEA_TA-35",
        "MEA_TOP40",
    ],
    "additional_data_path": "_all_assets_data.pkl.gz",
}

# problem params
problem_config = {
    "export_path": "./Results/",
    "val_period": VAL_PERIOD,  # if val is 0, then its results are the same as training
    "holdout_period": 756,
}

# model params
model_config = {
    "baseline": MODE,
    "buy_and_hold": {},
    "risk_parity": {"window": 252},
    "ts_mom": {"window": 252},
    "csec_mom": {"window": 252, "fraction": 0.33},
}

# pre-allocation
export_label = (
    "valperiod_"
    + str(problem_config["val_period"])
    + "_testperiod_"
    + str(problem_config["holdout_period"])
    + "_baseline_"
    + model_config["baseline"]
)
if DAYS_SHIFTED:
    export_label += "_shifted_" + str(DAYS_SHIFTED)

data_config["export_label"] = export_label
problem_config["export_label"] = export_label
model_config["export_label"] = export_label
model_config["export_path"] = problem_config["export_path"]


In [61]:
# get data
Xtrain_tasks, Xval_tasks, Xtest_tasks = get_data(data_config, problem_config)

Asia and Pacific Asia and Pacific_AS51 (4253, 86)
Asia and Pacific Asia and Pacific_FBMKLCI (4253, 22)
Asia and Pacific Asia and Pacific_HSI (4253, 37)
Asia and Pacific Asia and Pacific_JCI (4253, 44)
Asia and Pacific Asia and Pacific_KOSPI (4253, 297)
Asia and Pacific Asia and Pacific_KSE100 (4253, 41)
Asia and Pacific Asia and Pacific_NIFTY (4253, 37)
Asia and Pacific Asia and Pacific_NKY (4253, 186)
Asia and Pacific Asia and Pacific_NZSE50FG (4253, 19)
Asia and Pacific Asia and Pacific_PCOMP (4253, 16)
Asia and Pacific Asia and Pacific_STI (4253, 25)
Asia and Pacific Asia and Pacific_SHSZ300 (4253, 18)
Asia and Pacific Asia and Pacific_TWSE (4253, 227)
Europe Europe_AEX (4253, 17)
Europe Europe_ASE (4253, 51)
Europe Europe_ATX (4253, 13)
Europe Europe_BEL20 (4253, 14)
Europe Europe_BUX (4253, 8)
Europe Europe_BVLX (4253, 17)
Europe Europe_CAC (4253, 35)
Europe Europe_CYSMMAPA (4253, 42)
Europe Europe_DAX (4253, 25)
Europe Europe_HEX (4253, 57)
Europe Europe_IBEX (4253, 23)
Europe Eu

### Training

In [62]:
# set model
if model_config["baseline"] == "buy_and_hold":
    trad_strat = BuyAndHold(Xtrain_tasks, model_config)
    add_label = [""] * len(data_config["region"])

elif model_config["baseline"] == "risk_parity":
    trad_strat = RiskParity(Xtrain_tasks, model_config)
    add_label = [""] * len(data_config["region"])

elif model_config["baseline"] == "ts_mom":
    trad_strat = TimeSeriesMomentum(Xtrain_tasks, model_config)
    add_label = [""] * len(data_config["region"])

elif model_config["baseline"] == "csec_mom":
    trad_strat = CrossSectionalMomentum(Xtrain_tasks, model_config)
    add_label = [""] * len(data_config["region"])

# additional labelling
to_add_label = {}
for lab, region in zip(add_label, data_config["region"]):
    to_add_label[region] = lab

# train model
start = time()
trad_strat.train()
print(time() - start)

# get signals
Xtrain_signal = trad_strat.predict(Xtrain_tasks)
Xval_signal = trad_strat.predict(Xval_tasks)
Xtest_signal = trad_strat.predict(Xtest_tasks)

Baseline models - Training is done in predict()
0.0


C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\3297836661.py:247: RuntimeWarning: invalid value encountered in divide
  sum_abs == 0, 0.0, crosssectional_signal / sum_abs


### Evaluation

In [63]:
# compute results
k, g = True, True
for region in data_config["region"]:
    region_task_paths = [t + "_all_assets_data.pkl.gz" for t in data_config[region]]

    z = True
    for task, task_path in zip(data_config[region], region_task_paths):
        # get signal
        pred_train = Xtrain_signal[region][task][:-1, :]
        pred_val = Xval_signal[region][task][:-1, :]
        pred_test = Xtest_signal[region][task][:-1, :]

        # get target
        Ytrain = Xtrain_tasks[region][task][1:, :]
        Yval = Xval_tasks[region][task][1:, :]
        Ytest = Xtest_tasks[region][task][1:, :]

        # compute returns
        df_train_ret = np.multiply(pred_train, Ytrain) - compute_trading_costs(
            pred_train
        )
        df_val_ret = np.multiply(pred_val, Yval) - compute_trading_costs(pred_val)
        df_test_ret = np.multiply(pred_test, Ytest) - compute_trading_costs(pred_test)

        # get performance metrics
        df = pd.read_pickle(data_config["data_path"] + task_path)
        df_train_ret = pd.DataFrame(df_train_ret, columns=df.columns)
        df_train_metrics = compute_performance_metrics(df_train_ret)
        df_train_metrics["exchange"] = task

        df_val_ret = pd.DataFrame(df_val_ret, columns=df.columns)
        df_val_metrics = compute_performance_metrics(df_val_ret)
        df_val_metrics["exchange"] = task

        df_test_ret = pd.DataFrame(df_test_ret, columns=df.columns)
        df_test_metrics = compute_performance_metrics(df_test_ret)
        df_test_metrics["exchange"] = task

        if z:
            all_df_train_metrics = df_train_metrics.copy()
            all_df_val_metrics = df_val_metrics.copy()
            all_df_test_metrics = df_test_metrics.copy()
            z = False
        else:
            all_df_train_metrics = pd.concat(
                [all_df_train_metrics, df_train_metrics], axis=0
            )
            all_df_val_metrics = pd.concat([all_df_val_metrics, df_val_metrics], axis=0)
            all_df_test_metrics = pd.concat(
                [all_df_test_metrics, df_test_metrics], axis=0
            )

    # export results
    all_df_train_metrics["region"] = region
    all_df_train_metrics["set"] = "train"
    all_df_val_metrics["region"] = region
    all_df_val_metrics["set"] = "val"
    all_df_test_metrics["region"] = region
    all_df_test_metrics["set"] = "test"

    pd.concat(
        [all_df_train_metrics, all_df_val_metrics, all_df_test_metrics], axis=0
    ).to_csv(
        problem_config["export_path"]
        + region
        + "_"
        + problem_config["export_label"]
        + to_add_label[region]
        + ".csv"
    )

    # consolidate test results
    if g:
        global_df_test_metrics = all_df_test_metrics.copy()
        g = False
    else:
        global_df_test_metrics = pd.concat(
            [global_df_test_metrics, all_df_test_metrics.copy()], axis=0
        )

C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
  return annual_return(x).values / annual_volatility(x)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22320\2942208226.py:87: RuntimeWarning: invalid value encountered in divide
